### Импорт нужных библиотек: 

(я решил использовать sklearn)

In [3]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score, ndcg_score
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import LabelEncoder
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
import xgboost as xgb
import numpy as np
import pandas as pd


### Load the data: 
Подготовка данных: загружаю нужные датасеты и объединяю основной датасет с датасетом песен и пользователей, чтобы на основе этих данных (о пользователе и о песне) тренировать модель

In [4]:
train_data = pd.read_csv("train.csv")
train_data.head()

,msno,song_id,source_system_tab,source_screen_name,source_type,target
0,FGtllVqz18RPiwJj/edr2gV78zirAiY/9SmYvia+kCg=,BBzumQNXUHKdEBOB7mAJuzok+IJA1c2Ryg/yzTF6tik=,explore,Explore,online-playlist,1
1,Xumu+NIjS6QYVxDS4/t3SawvJ7viT9hPKXmf0RtLNx8=,bhp/MpSNoqoxOIB+/l8WPqu6jldth4DIpCm3ayXnJqM=,my library,Local playlist more,local-playlist,1
2,Xumu+NIjS6QYVxDS4/t3SawvJ7viT9hPKXmf0RtLNx8=,JNWfrrC7zNN7BdMpsISKa4Mw+xVJYNnxXh3/Epw7QgY=,my library,Local playlist more,local-playlist,1
3,Xumu+NIjS6QYVxDS4/t3SawvJ7viT9hPKXmf0RtLNx8=,2A87tzfnJTSWqD7gIZHisolhe4DMdzkbd6LzO1KHjNs=,my library,Local playlist more,local-playlist,1
4,FGtllVqz18RPiwJj/edr2gV78zirAiY/9SmYvia+kCg=,3qm6XTZ6MOCU11x8FIVbAGH5l5uMkT3/ZalWG1oo2Gc=,explore,Explore,online-playlist,1


In [5]:
members_data = pd.read_csv("members.csv")
members_data.head()

,msno,city,bd,gender,registered_via,registration_init_time,expiration_date
0,XQxgAYj3klVKjR3oxPPXYYFp4soD4TuBghkhMTD4oTw=,1,0,NaN,7,20110820,20170920
1,UizsfmJb9mV54qE9hCYyU07Va97c0lCRLEQX3ae+ztM=,1,0,NaN,7,20150628,20170622
2,D8nEhsIOBSoE6VthTaqDX8U6lqjJ7dLdr72mOyLya2A=,1,0,NaN,4,20160411,20170712
3,mCuD+tZ1hERA/o5GPqk38e041J8ZsBaLcu7nGoIIvhI=,1,0,NaN,9,20150906,20150907
4,q4HRBfVSssAFS9iRfxWrohxuk9kCYMKjHOEagUMV6rQ=,1,0,NaN,4,20170126,20170613


In [6]:
songs_data = pd.read_csv("songs.csv")
songs_data.head()

,song_id,song_length,genre_ids,artist_name,composer,lyricist,language
0,CXoTN1eb7AI+DntdU1vbcwGRV4SCIDxZu+YD8JP8r4E=,247640,465,張信哲 (Jeff Chang),董貞,何啟弘,3.0
1,o0kFgae9QtnYgRkVPqLJwa05zIhRlUjfF7O1tDw0ZDU=,197328,444,BLACKPINK,TEDDY| FUTURE BOUNCE| Bekuh BOOM,TEDDY,31.0
2,DwVvVurfpuz+XPuFvucclVQEyPqcpUkHR0ne1RQzPs0=,231781,465,SUPER JUNIOR,NaN,NaN,31.0
3,dKMBWoZyScdxSkihKG+Vf47nc18N9q4m58+b4e7dSSE=,273554,465,S.H.E,湯小康,徐世珍,3.0
4,W3bqWd3T+VeHFzHAUfARgW9AvVRaF4N5Yzm4Mr6Eo/o=,140329,726,貴族精選,Traditional,Traditional,52.0


In [7]:
train_df = pd.merge(train_data, songs_data, on='song_id', how='left')

train_df = pd.merge(train_df, members_data, on='msno', how='left')
train_df.head()

,msno,song_id,source_system_tab,source_screen_name,source_type,target,song_length,genre_ids,artist_name,composer,lyricist,language,city,bd,gender,registered_via,registration_init_time,expiration_date
0,FGtllVqz18RPiwJj/edr2gV78zirAiY/9SmYvia+kCg=,BBzumQNXUHKdEBOB7mAJuzok+IJA1c2Ryg/yzTF6tik=,explore,Explore,online-playlist,1,206471.0,359,Bastille,Dan Smith| Mark Crew,NaN,52.0,1,0,NaN,7,20120102,20171005
1,Xumu+NIjS6QYVxDS4/t3SawvJ7viT9hPKXmf0RtLNx8=,bhp/MpSNoqoxOIB+/l8WPqu6jldth4DIpCm3ayXnJqM=,my library,Local playlist more,local-playlist,1,284584.0,1259,Various Artists,NaN,NaN,52.0,13,24,female,9,20110525,20170911
2,Xumu+NIjS6QYVxDS4/t3SawvJ7viT9hPKXmf0RtLNx8=,JNWfrrC7zNN7BdMpsISKa4Mw+xVJYNnxXh3/Epw7QgY=,my library,Local playlist more,local-playlist,1,225396.0,1259,Nas,N. Jones、W. Adams、J. Lordan、D. Ingle,NaN,52.0,13,24,female,9,20110525,20170911
3,Xumu+NIjS6QYVxDS4/t3SawvJ7viT9hPKXmf0RtLNx8=,2A87tzfnJTSWqD7gIZHisolhe4DMdzkbd6LzO1KHjNs=,my library,Local playlist more,local-playlist,1,255512.0,1019,Soundway,Kwadwo Donkoh,NaN,-1.0,13,24,female,9,20110525,20170911
4,FGtllVqz18RPiwJj/edr2gV78zirAiY/9SmYvia+kCg=,3qm6XTZ6MOCU11x8FIVbAGH5l5uMkT3/ZalWG1oo2Gc=,explore,Explore,online-playlist,1,187802.0,1011,Brett Young,Brett Young| Kelly Archer| Justin Ebach,NaN,52.0,1,0,NaN,7,20120102,20171005


In [12]:
train_df.shape

(7377418, 18)

### Clean the data

Небольшая очистка (в итоге я ничего не очистил, тк это пока не надо)

In [9]:
def find_dirty_values(data):
    dtypes = pd.DataFrame(data.dtypes,columns=["Data Type"])
    dtypes["Unique Values"]=data.nunique().sort_values(ascending=True)
    dtypes["Null Values"]=data.isnull().sum()
    dtypes["% null Values"]=data.isnull().sum()/len(data)
    return dtypes.sort_values(by="Null Values" , ascending=False).style.background_gradient(cmap='YlOrRd',axis=0)
find_dirty_values(train_df)

,Data Type,Unique Values,Null Values,% null Values
lyricist,object,33888,3178798,0.430882
gender,object,2,2961479,0.401425
composer,object,76064,1675706,0.227140
source_screen_name,object,20,414804,0.056226
genre_ids,object,572,118455,0.016056
source_system_tab,object,8,24849,0.003368
source_type,object,12,21539,0.002920
language,float64,10,150,0.000020
song_length,float64,60266,114,0.000015
artist_name,object,40582,114,0.000015


### Parse the dates 
Преобразовать столбцы с датами в удобную форму, чтобы если что с ними можно было работать (но это в итоге не понадобилось)

In [13]:
def parse_date(df, date):
    df[date] = pd.to_datetime(df[date].astype('str'))
    return df


train_df = parse_date(train_df, 'registration_init_time')
train_df = parse_date(train_df, 'expiration_date')
train_df.head()

,msno,song_id,source_system_tab,source_screen_name,source_type,target,song_length,genre_ids,artist_name,composer,lyricist,language,city,bd,gender,registered_via,registration_init_time,expiration_date
0,8158,74679,1,7,6,1,206471.0,285,3277,14581,33888,52.0,1,0,2,7,2012-01-02,2017-10-05
1,17259,223479,3,8,4,1,284584.0,90,31960,76064,33888,52.0,13,24,0,9,2011-05-25,2017-09-11
2,17259,120758,3,8,4,1,225396.0,90,21372,45057,33888,52.0,13,24,0,9,2011-05-25,2017-09-11
3,17259,23707,3,8,4,1,255512.0,6,27439,36700,33888,-1.0,13,24,0,9,2011-05-25,2017-09-11
4,8158,33308,1,7,6,1,187802.0,2,4472,8485,33888,52.0,1,0,2,7,2012-01-02,2017-10-05


### Encode categorical data 
(feature engineering step) 

Преобразовать данные в полностью числовой вид, чтобы на них можно было обучать модель   

In [14]:
cat_cols = train_df.select_dtypes(include="O").columns
encoder = LabelEncoder()
for col in cat_cols:
    train_df[col] = encoder.fit_transform(train_df[col])

In [15]:
x = train_df.drop(columns={"registration_init_time", "expiration_date", "song_id", "msno"})
x.head()

,source_system_tab,source_screen_name,source_type,target,song_length,genre_ids,artist_name,composer,lyricist,language,city,bd,gender,registered_via
0,1,7,6,1,206471.0,285,3277,14581,33888,52.0,1,0,2,7
1,3,8,4,1,284584.0,90,31960,76064,33888,52.0,13,24,0,9
2,3,8,4,1,225396.0,90,21372,45057,33888,52.0,13,24,0,9
3,3,8,4,1,255512.0,6,27439,36700,33888,-1.0,13,24,0,9
4,1,7,6,1,187802.0,2,4472,8485,33888,52.0,1,0,2,7


In [16]:
x = x.dropna()
find_dirty_values(x)

,Data Type,Unique Values,Null Values,% null Values
source_system_tab,int64,9,0,0.000000
source_screen_name,int64,21,0,0.000000
source_type,int64,13,0,0.000000
target,int64,2,0,0.000000
song_length,float64,60266,0,0.000000
genre_ids,int64,573,0,0.000000
artist_name,int64,40582,0,0.000000
composer,int64,76064,0,0.000000
lyricist,int64,33888,0,0.000000
language,float64,10,0,0.000000


In [17]:
y = x.pop('target')

In [18]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=.27,random_state=42)

## Testing different models
Далее, я приступаю к тренировке и тестированию точности различных моделей

### Logistic regression

In [22]:
lr = LogisticRegression(max_iter=300, C=0.001, penalty="l2")
lr.fit(x_train, y_train)
train_pred = lr.predict(x_train)
test_pred = lr.predict(x_test)
lr_train_acc = ndcg_score([y_train], [train_pred], k=20)
lr_test_acc = ndcg_score([y_test], [test_pred], k=20)
print("Training Accuracy : ", lr_train_acc)
print("Test Accuracy : ", lr_test_acc)
print("Classification Report:\n", classification_report(y_train, train_pred))


Training Accuracy :  0.5220806101762194
Test Accuracy :  0.5213100442637821
Classification Report:
               precision    recall  f1-score   support

           0       0.52      0.43      0.47   2673001
           1       0.52      0.61      0.56   2712404

    accuracy                           0.52   5385405
   macro avg       0.52      0.52      0.52   5385405
weighted avg       0.52      0.52      0.52   5385405



### Naive Bayes 

In [19]:
NB = GaussianNB(var_smoothing=0.01)
NB.fit(x_train,y_train)
train_pred = NB.predict(x_train)
test_pred = NB.predict(x_test)
NB_train_acc = ndcg_score([y_train], [train_pred], k=20)
NB_test_acc = ndcg_score([y_test], [test_pred], k=20)
print("Training Accuracy : ",NB_train_acc)
print("Test Accuracy : ", NB_test_acc)
print("Classification Report:\n", classification_report(y_train,train_pred))


Training Accuracy :  0.5128118326333029
Test Accuracy :  0.5126159431005338
Classification Report:
               precision    recall  f1-score   support

           0       0.56      0.15      0.24   2673001
           1       0.51      0.88      0.65   2712404

    accuracy                           0.52   5385405
   macro avg       0.53      0.52      0.44   5385405
weighted avg       0.53      0.52      0.44   5385405



### Decision trees 

In [20]:
Dtree = DecisionTreeClassifier(max_depth=None, min_samples_leaf=1, min_samples_split=5)
Dtree.fit(x_train,y_train)
train_pred = Dtree.predict(x_train)
test_pred = Dtree.predict(x_test)
Dtree_train_acc = ndcg_score([y_train], [train_pred], k=20)
Dtree_test_acc = ndcg_score([y_test], [test_pred], k=20)
print("Training Accuracy : ",Dtree_train_acc)
print("Test Accuracy : ",Dtree_test_acc)
print("Classification Report:\n",classification_report(y_train,train_pred))



Training Accuracy :  0.8850151423287034
Test Accuracy :  0.619688715554485
Classification Report:
               precision    recall  f1-score   support

           0       0.86      0.89      0.87   2673001
           1       0.89      0.85      0.87   2712404

    accuracy                           0.87   5385405
   macro avg       0.87      0.87      0.87   5385405
weighted avg       0.87      0.87      0.87   5385405



### Gradient boosting

In [26]:
gb = GradientBoostingClassifier(learning_rate=.1, max_depth=4, n_estimators=50)
gb.fit(x_train, y_train)
train_pred = gb.predict(x_train)
test_pred = gb.predict(x_test)
gb_train_acc = ndcg_score([y_train], [train_pred], k=20)
gb_test_acc = ndcg_score([y_test], [test_pred], k=20)
print("Training Accuracy : ", gb_train_acc)
print("Test Accuracy : ", gb_test_acc)
print("Classification Report:\n", classification_report(y_train, train_pred))


Training Accuracy :  0.6359447238555372
Test Accuracy :  0.6367587203773442
Classification Report:
               precision    recall  f1-score   support

           0       0.62      0.65      0.63   2674489
           1       0.64      0.60      0.62   2711026

    accuracy                           0.63   5385515
   macro avg       0.63      0.63      0.63   5385515
weighted avg       0.63      0.63      0.63   5385515



### XG boost 

In [27]:
xg = xgb.XGBClassifier(random_state=0, learning_rate=.01, max_depth=3, n_estimators=50)
xg.fit(x_train, y_train)
train_pred = xg.predict(x_train)
test_pred = xg.predict(x_test)
xg_train_acc = ndcg_score([y_train], [train_pred], k=20)
xg_test_acc = ndcg_score([y_test], [test_pred], k=20)
print("Training Accuracy : ", xg_train_acc)
print("Test Accuracy : ", xg_test_acc)
print("Classification Report:\n", classification_report(y_train, train_pred))


Training Accuracy :  0.6401218661325457
Test Accuracy :  0.641066484452648
Classification Report:
               precision    recall  f1-score   support

           0       0.61      0.67      0.64   2674489
           1       0.64      0.58      0.61   2711026

    accuracy                           0.62   5385515
   macro avg       0.62      0.62      0.62   5385515
weighted avg       0.63      0.62      0.62   5385515



In [ ]:
clf = SVC()
clf.fit(x_train, y_train)
train_pred = clf.predict(x_train)
test_pred = clf.predict(x_test)
svc_train_acc = ndcg_score([y_train], [train_pred], k=20)
svc_test_acc = ndcg_score([y_test], [test_pred], k=20)
print("Training Accuracy : ", svc_train_acc)
print("Test Accuracy : ", svc_test_acc)
print("Classification Report:\n", classification_report(y_train, train_pred))
# у меня эта модель в итоге не запустилась, тк слишком слабый компьютер, ноя ее все равно тут оставил, чтобы была 

### Comparing models
Финальное сравнение точности моделей. Тут я решаю, какую модель в итоге буду использовать 

In [28]:
cols = [
    ["Logistic Regression", lr_train_acc, lr_test_acc],
    ["Naive Bayes", NB_train_acc, NB_test_acc],
    ["Decision Trees", Dtree_train_acc, Dtree_test_acc],
    ["Gradient Boosting", gb_train_acc, gb_test_acc],
    ["XGBoost", xg_train_acc, xg_test_acc]
    ["SVC", svc_train_acc, svc_test_acc]
    ]
results = pd.DataFrame(cols,
                       columns = ["Model","Training Accuracy %","Test Evaluation %"]).sort_values(
                        by="Test Evaluation %", ascending=False)
results.style.background_gradient(cmap='Set1')



,Model,Training Accuracy %,Test Evaluation %
2,Decision Trees,0.663774,0.657518
4,XGBoost,0.640122,0.641066
3,Gradient Boosting,0.635945,0.636759
0,Logistic Regression,0.594526,0.595026
1,Naive Bayes,0.540503,0.540784


Поскольку наивысшая точность оказалась у алгоритма дерева решений (decision trees classification algorithm), я решил использовать эту модель, чтобы предсказать релевантность песен для пользователей из датасета test.csv 

Далее я применяю эту модель к тестовому датасету и записываю результат в файл 

In [22]:
test_data = pd.read_csv('test.csv')

test_data = pd.merge(test_data, songs_data, on='song_id', how='left')
test_data = pd.merge(test_data, members_data, on='msno', how='left')

test_data = parse_date(test_data, 'registration_init_time')
test_data = parse_date(test_data, 'expiration_date')
test_data.head()

,msno,song_id,source_system_tab,source_screen_name,source_type,target,song_length,genre_ids,artist_name,composer,lyricist,language,city,bd,gender,registered_via,registration_init_time,expiration_date
0,8158,74679,1,7,6,1,206471.0,285,3277,14581,33888,52.0,1,0,2,7,2012-01-02,2017-10-05
1,17259,223479,3,8,4,1,284584.0,90,31960,76064,33888,52.0,13,24,0,9,2011-05-25,2017-09-11
2,17259,120758,3,8,4,1,225396.0,90,21372,45057,33888,52.0,13,24,0,9,2011-05-25,2017-09-11
3,17259,23707,3,8,4,1,255512.0,6,27439,36700,33888,-1.0,13,24,0,9,2011-05-25,2017-09-11
4,8158,33308,1,7,6,1,187802.0,2,4472,8485,33888,52.0,1,0,2,7,2012-01-02,2017-10-05


In [29]:
cat_cols = train_df.select_dtypes(include="O").columns
encoder = LabelEncoder()
for col in cat_cols:
    train_df[col] = encoder.fit_transform(train_df[col])
    
x_test = train_df.drop(columns={"registration_init_time", "expiration_date", "song_id", "msno", "genre_ids"})
x_test = x_test.dropna()
x_test.head()


,source_system_tab,source_screen_name,source_type,target,song_length,artist_name,composer,lyricist,language,city,bd,gender,registered_via
0,1,7,6,1,206471.0,3277,14581,33888,52.0,1,0,2,7
1,3,8,4,1,284584.0,31960,76064,33888,52.0,13,24,0,9
2,3,8,4,1,225396.0,21372,45057,33888,52.0,13,24,0,9
3,3,8,4,1,255512.0,27439,36700,33888,-1.0,13,24,0,9
4,1,7,6,1,187802.0,4472,8485,33888,52.0,1,0,2,7


In [1]:
test_pred = Dtree.predict(x_test)
test_data['target'] = test_pred * Dtree_train_acc

NameError: name 'Dtree' is not defined

In [4]:
result = test_data[['id', 'target']]
result.to_csv('result.csv')

NameError: name 'test_data' is not defined